In [25]:
from llama_index.core import (
    VectorStoreIndex, 
    SimpleDirectoryReader, 
    StorageContext, 
    ServiceContext, 
    load_index_from_storage
)
from llama_index.core.node_parser import SemanticSplitterNodeParser
from llama_index.embeddings.gemini import GeminiEmbedding
from llama_index.llms.groq import Groq
from llama_index.postprocessor.cohere_rerank import CohereRerank
import os
from dotenv import load_dotenv
load_dotenv()

True

In [26]:
GOOGLE_API_KEY = os.getenv("GEMINI_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
COHERE_API_KEY = os.getenv("COHERE_API_KEY")

In [27]:
reader = SimpleDirectoryReader(input_dir="/Users/benwalsh/Desktop/Machine Learning/data")
documents = reader.load_data()

Generating embeddings:  22%|██▏       | 339/1575 [1:49:18<6:38:32, 19.35s/it]


In [28]:
embed_model = GeminiEmbedding(
    model_name="models/embedding-001", api_key=GOOGLE_API_KEY
)
splitter = SemanticSplitterNodeParser(
    buffer_size=1, 
    breakpoint_percentile_threshold=95, 
    embed_model=embed_model
)
nodes = splitter.get_nodes_from_documents(documents, show_progress=True)

Parsing nodes: 100%|██████████| 380/380 [22:14<00:00,  3.51s/it]


In [29]:
llm = Groq(model="mixtral-8x7b-32768", api_key=GROQ_API_KEY)

In [30]:
service_context = ServiceContext.from_defaults(embed_model=embed_model, llm=llm)

/var/folders/zd/94cpxbhj3pb1ks5h3qv9cm900000gn/T/ipykernel_18782/59880135.py:1: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embed_model, llm=llm)


In [31]:
vector_index = VectorStoreIndex.from_documents(documents, show_progress=True, 
               service_context=service_context, node_parser=nodes)

Generating embeddings: 100%|██████████| 388/388 [01:18<00:00,  4.94it/s]


In [32]:
vector_index.storage_context.persist(persist_dir="./storage")

In [33]:
storage_context = StorageContext.from_defaults(persist_dir="./storage")

In [34]:
index = load_index_from_storage(storage_context, service_context=service_context)

In [35]:
cohere_rerank = CohereRerank(api_key=COHERE_API_KEY, top_n=2)

In [36]:
query_engine = index.as_query_engine(service_context=service_context,
                similarity_top_k=10,
                node_postprocessors=[cohere_rerank],)

In [41]:
query = "How do I succeed in system programming?"
resp = query_engine.query(query)

unknown field: parameter model is not a valid field


In [42]:
resp.response

'To succeed in system programming, it is important to understand the concepts of writing code that interacts with the operating system to perform functions such as accessing peripherals, managing resources, and providing support for common functions. Familiarizing yourself with the interfaces presented by the operating system for tasks like memory management, creating tasks or threads, and coordinating and synchronizing tasks will also be crucial. Additionally, being meticulous in your coding practices, thoroughly testing your code, and being aware of potential issues like heap corruption can contribute to your success in system programming.'